# env

In [1]:
import os, sys, logging, json, pandas as pd, numpy as np
from glob import glob
from tqdm import tqdm
from multiprocessing import cpu_count
print(cpu_count())

import warnings
warnings.filterwarnings("ignore")
PROJECT = "yaa"
                                          
#WORKDIR = os.path.abspath(f"{PATH TO CODE}/{PROJECT}")
WORKDIR = os.path.abspath("./")
os.chdir(WORKDIR)

OUTPUTDIR=os.path.join(WORKDIR, "data")
DATADIR = os.path.join(WORKDIR, 'data')
NUM = int(1e16)

BATCHSIZE = 512

sys.path.insert(0, WORKDIR)
os.environ['PYTHONPATH']=WORKDIR


os.chdir(os.path.join(WORKDIR, PROJECT))
import util
from importlib import reload
reload(util)
args = util.parser.parse_args([])
util.set_logger(logging.INFO)

24


# gen rewrite

In [ ]:
modelid="Qwen/Qwen2-7B-Instruct"
for i in range(10):
    GENSEED=100+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/qw2_7b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}

In [ ]:
modelid="Qwen/Qwen2.5-7B-Instruct"
for i in range(10):
    GENSEED=200+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/qw2d5_7b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}

In [ ]:
modelid="mistralai/Mistral-7B-Instruct-v0.3"
for i in range(10):
    GENSEED=300+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/mistral0d3_7b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}

In [ ]:
modelid="google/gemma-2-9b-it"
for i in range(10):
    GENSEED=3000+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/gemma2_9b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}
    

In [ ]:
modelid="unsloth/llama-3-8b-Instruct"
for i in range(0, 10):
    GENSEED=1000+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/llama3_8b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}

In [ ]:
modelid="unsloth/Meta-Llama-3.1-8B-Instruct"
for i in range(10):
    GENSEED=2000+i
    !TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen   -modelid "{modelid}" -output_dir ../data/gen/llama3d1_8b_{GENSEED} -n_sample 1 -gen_seed {GENSEED}
    

# stage1

## AR_qw2d5_3b_d04

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_qw2d5_3b_d04"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Qwen2.5-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=212345
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_qw2d5_3b_d04"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Qwen2.5-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=212345
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1 2 3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "0 1 2 3" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## AR_llama3d2_1b_d37

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_d37"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_d37"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1 2 3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "0 1 2 3" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## AR_llama3d2_3b_d03

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_d03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_d03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_d03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_d03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "0 1 2 3" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## AR_gemma2_2b_d02

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_d02"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_d02"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_d02"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_d02"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 16 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## SFT_debv2xxl_d36

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_d36"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=345342
DATASEED=SEED
KN=4
KFIDS=0





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs  15  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0.4 -bi_rdrop 2  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_d36"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=345342
DATASEED=SEED
KN=4
KFIDS=1





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs  15  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0.4 -bi_rdrop 2  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_d36"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=345342
DATASEED=SEED
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs  15  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0.4 -bi_rdrop 2  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "0 1 2 3" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_d36"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=345342
DATASEED=SEED
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs  15  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0.4 -bi_rdrop 2  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

## SFT_debv2xl_d120

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_d120"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=19627
DATASEED=SEED
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.4 -avg_pool -bi_rdrop 3 \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_d120"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=19627
DATASEED=SEED
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.4 -avg_pool -bi_rdrop 3 \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_d120"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=19627
DATASEED=SEED
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.4 -avg_pool -bi_rdrop 3 \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_d120"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=19627
DATASEED=SEED
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.4 -avg_pool -bi_rdrop 3 \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


## SPLIT_debv2xl_d10

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_d10"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS=0





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.1 -avg_pool -bi_rdrop 3  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_d10"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.1 -avg_pool -bi_rdrop 3  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_d10"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.1 -avg_pool -bi_rdrop 3  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_d10"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 3 \
-epochs 15   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.1 -weight_decay 0.01 -bs 1 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0.1 -avg_pool -bi_rdrop 3  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



# gen semi

## gen data  (please run the cell sequentially, the program will exclude already used examples)

### gen2

In [ ]:


modelid="unsloth/Meta-Llama-3.1-8B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen2   -modelid "{modelid}" -output_dir ../data/gen2 -prefix llama3d1_8b -n_sample 1 -n_gen 6000

In [ ]:

modelid="unsloth/llama-3-8b-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen2   -modelid "{modelid}" -output_dir ../data/gen2 -prefix llama3_8b -n_sample 1 -n_gen 6000

In [ ]:
modelid="mistralai/Mistral-7B-Instruct-v0.3"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen2   -modelid "{modelid}" -output_dir ../data/gen2 -prefix mistral0d3_7b -n_sample 1 -n_gen 6000

In [ ]:
modelid="Qwen/Qwen2-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen2   -modelid "{modelid}" -output_dir ../data/gen2 -prefix qw2_7b -n_sample 1 -n_gen 6000

In [ ]:
modelid="Qwen/Qwen2.5-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen2   -modelid "{modelid}" -output_dir ../data/gen2 -prefix qw2d5_7b -n_sample 1 -n_gen 6000

### gen3

In [ ]:

modelid="Qwen/Qwen2-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen3 -prefix qw2_7b -n_sample 1 -temperature 1 -n_gen 6000

In [ ]:

modelid="Qwen/Qwen2.5-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen3 -prefix qw2d5_7b -n_sample 1 -temperature 1 -n_gen 6000

In [ ]:

modelid="mistralai/Mistral-7B-Instruct-v0.3"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen3 -prefix mistral0d3_7b -n_sample 1 -temperature 1 -n_gen 6000

In [ ]:

modelid="unsloth/llama-3-8b-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen3 -prefix llama3_8b -n_sample 1 -temperature 1 -n_gen 6000

In [ ]:
modelid="unsloth/Meta-Llama-3.1-8B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen3 -prefix llama3d1_8b -n_sample 1 -temperature 1 -n_gen 6000

### gen4

In [ ]:
modelid="microsoft/Phi-3-small-8k-instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen4 -prefix phi3_small -n_sample 1 -temperature 1 -n_gen 6000 -trust_remote_code

In [ ]:

modelid="01-ai/Yi-1.5-9B-Chat"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen4 -prefix yi1d5_9b -n_sample 1 -temperature 1 -n_gen 6000 -trust_remote_code

In [ ]:
modelid="Qwen/Qwen2-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen4 -prefix qw2_7b -n_sample 1 -temperature 1 -n_gen 6000

### gen5

In [ ]:
modelid="Qwen/Qwen2-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix qw2_7b -n_sample 1 -temperature 1 -n_gen 30000

In [ ]:
modelid="Qwen/Qwen2.5-7B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix qw2d5_7b -n_sample 1 -temperature 1 -n_gen 30000

In [ ]:

modelid="mistralai/Mistral-7B-Instruct-v0.3"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix mistral0d3_7b -n_sample 1 -temperature 1 -n_gen 30000

In [ ]:
modelid="unsloth/llama-3-8b-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix llama3_8b -n_sample 1 -temperature 1 -n_gen 30000

In [ ]:
modelid="unsloth/Meta-Llama-3.1-8B-Instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix llama3d1_8b -n_sample 1 -temperature 1 -n_gen 30000

In [ ]:
modelid="microsoft/Phi-3-small-8k-instruct"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix phi3_small -n_sample 1 -temperature 1 -n_gen 30000 -trust_remote_code

In [ ]:

modelid="01-ai/Yi-1.5-9B-Chat"
!TRANSFORMERS_OFFLINE=1 python gen_data.py -m gen3   -modelid "{modelid}" -output_dir ../data/gen5 -prefix yi1d5_9b -n_sample 1 -temperature 1 -n_gen 30000 -trust_remote_code

## predict

### predict gen3

In [ ]:

model_names = "AR_qw2d5_3b_d04"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

In [ ]:
model_names = "AR_gemma2_2b_d02"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

In [ ]:
model_names = "AR_llama3d2_3b_d03"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

In [ ]:
model_names = "AR_llama3d2_1b_d37"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

In [ ]:
model_names = "SPLIT_debv2xl_d10 SFT_debv2xl_d120 SFT_debv2xxl_d36"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 2  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

### predict gen5

In [ ]:
model_names = "AR_qw2d5_3b_d04"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen5 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen5

In [ ]:
model_names = "AR_llama3d2_3b_d03"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen5 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen5

In [ ]:
model_names = "AR_llama3d2_1b_d37"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen5 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen5

In [ ]:
model_names = "AR_gemma2_2b_d02"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen5 -vbs 1  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen5

In [ ]:
model_names = "SPLIT_debv2xl_d10 SFT_debv2xl_d120 SFT_debv2xxl_d36"
for model_name in model_names.split():
    
    !python eval.py  -kn 4 -kfids "0 1 2 3" -data_type test  -ds gen3 -vbs 2  -model_name "{model_name}" -dataloader_num_workers 4 \
    -torch_dtype float16 -use_fp16 -do_test  -suffix gen3

## gen semi label

###  gen3

In [ ]:
import util
DS='gen3'
args = util.parser.parse_args([])
data = util.load_gen2_data(args, data_dir=DS)
model_names = "SPLIT_debv2xl_d10 SFT_debv2xl_d120 SFT_debv2xxl_d36  AR_gemma2_2b_d02 AR_llama3d2_3b_d03 AR_llama3d2_1b_d37 AR_qw2d5_3b_d04"
step = 0.05
#0.8425971893444323
model_weights = search_model_weight_func(model_names, step=step)


model_names = {model_name:[f"{model_name}_KF{i}" for i in range(4)] for model_name in model_names.split()}
preds = load_model_preds(model_names, model_weights, suffix=DS)
preds = data.merge(preds, on='uid')
for i, col in enumerate(util.binary_labels):
    preds[col] = preds.pred.apply(lambda x: x[i])
preds['InjuryLocationType'] = preds.pred.apply(lambda x: x[len(util.binary_labels):len(util.binary_labels)+util.n_InjuryLocationType])
preds['WeaponType1'] =preds.pred.apply(lambda x: x[-util.n_WeaponType1:])
preds['src'] = f'semi_{DS}'


util.dump(preds, f'../data/semi_{DS}.dump')      
print('DONE')

### gen5

In [ ]:
import util
DS='gen5'
args = util.parser.parse_args([])
args.dataset = DS
data = util.load_data(args)
model_names = "SPLIT_debv2xl_d10 SFT_debv2xl_d120 SFT_debv2xxl_d36  AR_gemma2_2b_d02 AR_llama3d2_3b_d03 AR_llama3d2_1b_d37 AR_qw2d5_3b_d04"
step = 0.05
#0.8425971893444323
model_weights = search_model_weight_func(model_names, step=step)


model_names = {model_name:[f"{model_name}_KF{i}" for i in range(4)] for model_name in model_names.split()}
preds = load_model_preds(model_names, model_weights, suffix=DS)
preds = data.merge(preds, on='uid')
for i, col in enumerate(util.binary_labels):
    preds[col] = preds.pred.apply(lambda x: x[i])
preds['InjuryLocationType'] = preds.pred.apply(lambda x: x[len(util.binary_labels):len(util.binary_labels)+util.n_InjuryLocationType])
preds['WeaponType1'] =preds.pred.apply(lambda x: x[-util.n_WeaponType1:])
preds['src'] = f'semi_{DS}'


util.dump(preds, f'../data/semi_{DS}.dump')      
print('DONE')

# stage 2

## AR_llama3d2_1b_semid05

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_semid05"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_semid05"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_semid05"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_1b_semid05"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-1B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=543509
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## AR_llama3d2_3b_semid01

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_llama3d2_3b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/Llama-3.2-3B-Instruct"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=232567
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head  \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## AR_gemma2_2b_semid01

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="AR_gemma2_2b_semid01"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="unsloth/gemma-2-2b-it"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=905687
DATASEED=SEED
TRAINCOLS=" ".join(util.all_labels)
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 2   -max_seq_len 8192  -do_train  -dataloader_num_workers 4 -es 1 -verbose 320 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 2e-4 -warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 1 -torch_dtype bfloat16 -use_bf16 -train_cols {TRAINCOLS}  \
-use_unsloth -use_lora -lora_dropout 0.05 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj lm_head -gradient_checkpointing \
-use_ppt2 -aug_order 0.5 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 1  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval \
2>&1 | tee -a "{LOGFILE}"



## SFT_debv2xxl_semid03

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_semid03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=345342
DATASEED=SEED
KN=4
KFIDS=0





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs  10  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 64 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0 -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_semid03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=504930
DATASEED=345342
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs  10  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 64 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0 -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_semid03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=504930
DATASEED=345342
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs  10  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 64 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0 -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xxl_semid03"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xxlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=546455
DATASEED=345342
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs  10  -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 64 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing -avg_pool \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_mix 0 -aug_lower 0 -aug_text 0 -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

## SFT_debv2xl_semid11

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_semid11"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=432543
DATASEED=19627
KN=4
KFIDS="0"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 2 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -bi_rdrop 1  -avg_pool -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_semid11"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=432543
DATASEED=19627
KN=4
KFIDS="1"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 2 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -bi_rdrop 1  -avg_pool -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_semid11"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=432543
DATASEED=19627
KN=4
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 2 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -bi_rdrop 1  -avg_pool -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SFT_debv2xl_semid11"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=343902
DATASEED=19627
KN=4
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 2 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify  -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -bi_rdrop 1  -avg_pool -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump"  \
2>&1 | tee "{LOGFILE}"


!python eval.py  -kn {KN} -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval


## SPLIT_debv2xl_semid06

In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_semid06"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS=0





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0 -avg_pool -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_semid06"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS=1





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0 -avg_pool -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "0 1 2 3" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_semid06"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS="2"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 2 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0 -avg_pool -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval



In [ ]:

os.chdir(os.path.join(WORKDIR, PROJECT))

import util
LOGDIR=f"{DATADIR}/logs"
!mkdir -p {LOGDIR}
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)


MODELNAME="SPLIT_debv2xl_semid06"
DS="yaa"
DSCLS="Dataset"
VDSCLS="Dataset"
RESTORE=""
BACKBONE="microsoft/deberta-v2-xlarge"
# BACKBONE=sorted(glob(f"../data/{RESTORE}/checkpoint-*))[-1]
SEED=985034
DATASEED=SEED
KFIDS="3"





!nvidia-smi


!python -u sft.py -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -kn 4  -kfids {KFIDS} -backbone "{BACKBONE}" -data_seed {DATASEED} -seed {SEED} -eval_delay 0 \
-epochs 10   -max_seq_len 8192  -do_train -do_eval -dataloader_num_workers 4 -es 3 -verbose 32 -disable_tqdm -save_strategy epoch -evaluation_strategy epoch  -save_total_limit 1 \
-lr 5e-5 -warmup_ratio 0.05 -weight_decay 0.01 -bs 2 -gas 16 -vbs 2 -torch_dtype bfloat16 -use_bf16 -is_classify -is_split -use_full -gradient_checkpointing \
-w_bi 0.33333 -w_lt 0.33333 -w_wt 0.33333  -aug_order 0 -aug_missing 0 -aug_mix 0 -aug_text 0 -avg_pool -bi_rdrop 1 -semi_ratio 1 -semi_fpath "../data/semi_gen3.dump ../data/semi_gen5.dump" \
2>&1 | tee "{LOGFILE}"

!python eval.py  -kn 4 -kfids "{KFIDS}" -data_type train  -ds {DS} -vbs 2  -model_name {MODELNAME} -dataloader_num_workers 4 \
-torch_dtype float16 -use_fp16 -do_eval

